<a href="https://colab.research.google.com/github/ConorD28/NBA-Research/blob/main/NBA_Before_Finals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.decomposition import PCA
%matplotlib inline
df = pd.read_csv('NBA Finals Research_Upload.csv')
#df.drop(df.tail(37).index,
#        inplace = True)
df2 = df.drop(columns=df.columns[0], axis=1)
df_NP = df2.drop(columns=df.columns[-25:], axis=1)

In [ ]:
df_NP

,WinPerc_Per 100 Plays,PTSPer 100 Plays,PTS Above AveragePer 100 Plays,FGPercPer 100 Plays,3PPercPer 100 Plays,FTAPer 100 Plays,FTPercPer 100 Plays,OREBPer 100 Plays,DREBPer 100 Plays,ASTPer 100 Plays,...,ASTPerTOPerSOS_Per 100_Clutch_Plr,AST RATIOPerSOS_Per 100_Clutch_Plr,OREBPercPerSOS_Per 100_Clutch_Plr,DREBPercPerSOS_Per 100_Clutch_Plr,TO RATIOPerSOS_Per 100_Clutch_Plr,EFGPercPerSOS_Per 100_Clutch_Plr,TSPercPerSOS_Per 100_Clutch_Plr,USGPercPerSOS_Per 100_Clutch_Plr,PACEPerSOS_Per 100_Clutch_Plr,PIEPerSOS_Per 100_Clutch_Plr
0,0.6,103.3,-11.5,50.4,37.9,19.9,75.1,9.0,29.4,25.8,...,3.0,23.7,7.3,21.1,7.9,54.5,64.9,30.6,101.1,26.6
1,0.5,99.2,-15.6,46.0,34.4,20.8,83.1,8.8,28.0,21.6,...,2.5,11.5,6.0,12.6,4.2,46.9,55.2,32.4,91.3,20.9
2,0.6,99.3,-12.7,46.9,36.4,18.2,76.9,8.7,32.0,24.3,...,1.8,10.1,0.7,11.2,3.8,27.1,31.4,25.7,64.8,10.6
3,0.6,100.2,-11.8,46.6,35.6,18.7,81.6,9.4,31.9,22.3,...,0.4,4.5,4.2,13.8,8.5,29.6,39.2,25.1,77.5,10.1
4,0.7,100.4,-10.0,47.4,36.6,19.3,80.4,8.4,31.2,22.3,...,1.2,7.8,4.9,12.1,6.0,48.0,53.8,36.2,88.2,22.2
5,0.7,102.8,-7.6,49.1,38.5,17.8,80.1,8.5,31.8,25.7,...,1.6,10.3,1.0,9.9,4.1,28.8,36.1,21.1,64.7,12.7
6,0.7,102.1,-6.5,50.3,39.1,18.3,81.5,7.6,31.6,26.4,...,0.8,9.9,1.8,11.4,9.0,47.6,51.8,23.0,77.3,15.1
7,0.6,101.0,-7.6,47.6,37.2,21.2,77.9,7.7,30.7,21.3,...,2.2,19.6,7.5,32.4,11.0,76.5,79.3,56.0,126.3,44.0
8,0.8,102.8,-6.0,49.5,38.3,20.0,78.8,8.3,31.1,26.9,...,1.3,2.7,0.8,12.3,1.4,29.5,34.1,16.2,63.0,12.1
9,0.6,100.6,-8.2,47.0,38.4,21.3,74.8,8.5,31.4,20.7,...,2.4,21.3,4.5,26.7,9.0,64.4,67.3,30.8,102.5,24.6


In [ ]:
df2.isnull().sum().sum() #Check if there are NA values

0

In [ ]:
import scipy.stats
def correlation(dataset, threshold, target): #Function to get Pearson's correlation between input and target
  data = []
  for i in range(len(dataset.columns)):
      cor2 = dataset.iloc[:,i].corr(target) #scipy.stats.spearmanr(x, y)[0] and scipy.stats.kendalltau(x, y)[0] or Permutation Importance
      #cor2 = scipy.stats.spearmanr(dataset.iloc[:,i], target[10:30])[0]
      #cor2 = scipy.stats.kendalltau(dataset.iloc[:,i], target[10:30])[0]
      column_headers = list(dataset.columns.values)
      if(abs(cor2) > threshold):
        data.append(dataset.iloc[:,i]) #make list of columns that meet the threshold
        #if(abs(cor2) < .7):
         # print(column_headers[i], f'cor:{cor2:.3f}')
      i = i + 1
  return data

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split
from numpy.random.mtrand import random_sample
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV, MultiTaskLassoCV
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
def Scores(y, y_pred, y_full):
  MSE = mean_squared_error(y, y_pred)
  MAE = mean_absolute_error(y, y_pred)
  Normalized_RMSE = (np.sqrt(MSE)/np.mean(y_full))*100
  Normalized_MAE = (MAE/np.mean(y_full))*100
  Avg_Normalized_Score = (Normalized_RMSE + Normalized_MAE)/2
  avg_error = (sum(abs(y_pred-y)))/10
  print(f'Avg. Normalized Score:{ Avg_Normalized_Score:.1f}%')
  print(f'Normalized RMSE:{ Normalized_RMSE:.1f}%')
  print(f'Normalized MAE:{ Normalized_MAE:.2f}%')
  #print(f'MAE:{ MAE:.3f}')
  #print(f'RMSE:{ np.sqrt(MSE):.3f}')
  print(f'Avg. Error:{avg_error:.1f}')
  print(abs(y_pred-y))
  return Avg_Normalized_Score

In [ ]:
def RLE_Model(X, y, choice, predict_df): #Function to run Ridge, Lasso, or ElasticNet model
  #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0) #Train/Test
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]

  if(choice=="Ridge"):
    alphas = np.geomspace(1e-10, 1e10, num=100)
    m1 = RidgeCV(alphas=alphas)
    m1.fit(X_train, y_train)
    print()

  if(choice=="Lasso"):
    alphas = np.geomspace(1e-10, 1e10, num=100)
    m1 = LassoCV(alphas=alphas)
    m1.fit(X_train, y_train)
    print("Lasso alpha: ", m1.alpha_)

  if(choice=="Elastic"):
    l1_ratio = [0, 0.3, 0.5, 0.7, 0.9, 1]
    alphas = np.geomspace(1e-10, 1e10, num=100)
    m1 = ElasticNetCV(alphas=alphas, l1_ratio=l1_ratio, max_iter=100000)
    m1.fit(X_train, y_train)
    print("Elastic alpha: ", m1.alpha_)

  #print(f'Chosen alpha  {pipeline.steps[0][1].alpha_:.6f}')
  #print(f'Intercept (b) {pipeline.steps[0][1].intercept_:.6f}')
  #print(pd.Series(pipeline.steps[0][1].coef_, index=X.columns),'\n')

  #Calculate the predicted values:
  y_train_pred = m1.predict(X_train)
  #print(y_train_pred)
  print()

  y_test_pred = m1.predict(X_test)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Test Predictions:
  print("Test predictions:")
  print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.2f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  #Predict:
  predictions = m1.predict(predict_df)

  return y_test_pred, y_train_pred, predictions

In [ ]:
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

def GBR_model(X,y, t, l, n, predict_df):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  reg = GradientBoostingRegressor(tol = t, learning_rate = l, n_estimators=n, random_state=0) #default: tol = 0.0001, learning rate - 0.1, 100, friedman_mse
  reg.fit(X_train, y_train)
  y_train_pred = reg.predict(X_train)
  #print(y_train_pred)

  y_test_pred = reg.predict(X_test)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Predictions:
  print("Test predictions:")
  print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.2f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  #Predict:
  predictions = reg.predict(predict_df)
  return y_test_pred, y_train_pred, predictions

In [ ]:
from sklearn.ensemble import BaggingRegressor
import xgboost as xgb
import pickle

def BR_model(X,y):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  reg = BaggingRegressor(base_estimator=xgb.XGBRegressor())
  #reg = pickle.load(open("BR_model2", "rb"))
  reg.fit(X_train, y_train)
  y_train_pred = reg.predict(X_train)
  #print(y_train_pred)

  y_test_pred = reg.predict(X_test)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Predictions:
  print("Test predictions:")
  print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.2f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  #Predict:
  #predictions = reg.predict(predict_df)
  #print(predictions)

  pickle.dump(reg, open("BR_model", "wb"))
  return y_test_pred, y_train_pred

In [ ]:
from sklearn.linear_model import SGDRegressor
#SGD Regressor:
def SGD_model(X,y, t, ep, predict_df):

  reg = SGDRegressor(max_iter=1000, tol=t, epsilon = ep) #tol = 0.001, epsilon=0.1
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  reg.fit(X_train, y_train)
  y_train_pred = reg.predict(X_train)
  #print(y_train_pred)

  y_test_pred = reg.predict(X_test)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Predictions:
  print("Test predictions:")
  print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.2f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  predictions = reg.predict(predict_df)
  #print(predictions)

  pickle.dump(reg, open("SGD_model", "wb"))
  return y_test_pred, y_train_pred, predictions

In [ ]:
#Keras Sequential Neural Net
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=2)

def Keras_model(X,y,e, u, u2, u3, u4, u5, choice):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  model = Sequential()
  model.add(Dense(u, input_dim=X_train.shape[1], activation='relu')) # Hidden 1, 60
  model.add(Dense(units=u2,activation='relu')) # Hidden 2, 30
  model.add(Dense(units=u3,activation='relu'))
  model.add(Dense(units=u4,activation='relu'))
  model.add(Dense(units=u5,activation='relu'))
  model.add(Dense(units=15,activation='relu')) #15
  model.add(Dense(units=1)) #,activation='relu'
  model.compile(loss='mean_squared_error', optimizer=choice) #
  model.fit(X_train, y_train, verbose=0, epochs=e, callbacks=[early_stop]); #callbacks=[early_stop]

  y_train_pred = model.predict(X_train)
  y_train_pred = y_train_pred.flatten()

  y_test_pred = model.predict(X_test)
  y_test_pred = y_test_pred.flatten()

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Predictions:
  print("Test predictions:")
  print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  #print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.2f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  model.save('/content/drive/MyDrive/Models/Keras_Model', save_format="h5")

In [ ]:
from sklearn.tree import DecisionTreeRegressor

def DTR_model(X,y,leafs):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  # We introduce regularization by increasing the value of min_samples_leaf
  tree_reg_regularized = DecisionTreeRegressor(random_state=42, min_samples_leaf=leafs)
  tree_reg_regularized.fit(X_train, y_train)
  y_train_pred = tree_reg_regularized.predict(X_train) #_regularized
  #print(y_train_pred)

  y_test_pred = tree_reg_regularized.predict(X_test) #_regularized

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Predictions:
  print("Test predictions:")
  print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.2f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

In [ ]:
from sklearn.svm import LinearSVR

def SVM_model(X,y,ep):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  svm_reg = LinearSVR(epsilon=ep, random_state=42) #default: epsilon = 0 tol=0.0001, C=1.0
  svm_reg.fit(X_train, y_train)

  #Train Predictions:
  y_train_pred = svm_reg.predict(X_train)
  #print(y_train_pred)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Test Predictions:
  y_test_pred = svm_reg.predict(X_test)
  print("Test predictions:")
  #print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.2f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

In [ ]:
from sklearn.svm import SVR

def SVM_models(X,y, choice, ep, C_value, predict_df):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  if(choice=="rbf"):
    model = SVR(kernel="rbf", C=C_value, gamma=0.1, epsilon=ep) #0.1 default ep; 100 default C, 0.1 default gamma
    model.fit(X_train, y_train)

  if(choice=="poly"):
    model = SVR(kernel="poly", C=C_value, gamma="auto", degree=3, epsilon=ep, coef0=1) #0.1 default ep; 100 default C
    model.fit(X_train, y_train)

  if(choice == "linear"):
    model = SVR(kernel="linear", C=C_value, gamma="auto", degree=3, epsilon=ep, coef0=1) #0.1 default ep; 100 default C
    model.fit(X_train, y_train)

  #Train Predictions:
  y_train_pred = model.predict(X_train)
  #print(y_train_pred)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Test Predictions:
  y_test_pred = model.predict(X_test)
  print("Test predictions:")
  print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.2f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  final_preds = model.predict(predict_df)

  return y_test_pred, y_train_pred, final_preds

In [ ]:
from sklearn.datasets import make_swiss_roll
from sklearn.manifold import LocallyLinearEmbedding

In [ ]:
df.iloc[:, 1110:]

,PTS_Per 100_Pfs,OPP PTS_Per 100_Pfs,Opp PtsPerPoints_Per 100_Pfs,Point Diff_Per 100_Pfs,+Per__Per 100_Pfs,OFFRTG_Per 100_Pfs,DEFRTG_Per 100_Pfs,Def RtgPerOff Rtg_Per 100_Pfs,NETRTG_Per 100_Pfs,PTS_PerGM_Pfs,...,EST_OFFRTG_Pfs,EST. OFFRTG Above AVG_Pfs,EST_DEFRTG_Pfs,EST. DEFRTG Below AVG_Pfs,EST_NETRTG_Pfs,PTS_Per 100_Pfs_Plr,PTS_PerGM_Pfs_Plr,EST_OFFRTG_Pfs_Plr,EST_DEFRTG_Pfs_Plr,EST_NETRTG_Pfs_Plr
0,103.7,96.2,0.9,7.5,8.3,118.2,110.2,0.9,8.0,113.5,...,116.7,1.9,108.0,6.8,8.7,33.2,30.0,118.2,109.2,9.1
1,100.2,98.8,1.0,1.4,1.5,113.8,111.9,1.0,1.9,108.3,...,110.5,-4.3,110.3,4.5,0.2,29.9,26.9,108.7,111.4,-2.7
2,101.0,96.4,1.0,4.6,4.6,114.5,109.6,1.0,4.9,111.9,...,112.4,0.4,107.3,4.7,5.1,33.7,27.4,113.9,107.6,6.3
3,97.5,94.2,1.0,3.3,3.3,110.3,106.3,1.0,3.9,105.5,...,108.1,-3.9,103.9,8.1,4.2,27.4,25.6,108.4,103.5,4.9
4,98.4,93.4,0.9,5.0,5.0,109.8,104.2,0.9,5.6,106.6,...,107.9,-2.5,101.4,9.0,6.5,34.5,30.5,111.2,102.0,9.2
5,100.6,97.6,1.0,3.0,3.0,114.6,111.5,1.0,3.1,114.1,...,112.3,1.9,108.6,1.8,3.7,37.0,32.3,115.9,106.6,9.3
6,100.7,91.6,0.9,9.1,9.1,112.7,102.0,0.9,10.7,110.4,...,111.9,3.3,100.5,8.1,11.4,33.2,29.0,114.9,101.1,13.8
7,95.1,96.8,1.0,-1.7,-1.6,108.3,111.1,1.0,-2.8,101.1,...,106.5,-2.1,108.5,0.1,-2.0,37.1,34.0,108.4,109.6,-1.2
8,105.8,93.8,0.9,12.0,12.0,118.2,105.3,0.9,12.9,119.3,...,116.3,7.5,102.8,6.0,13.6,34.0,28.5,122.6,106.6,16.0
9,107.2,99.9,0.9,7.3,7.3,119.9,111.8,0.9,8.1,116.2,...,118.1,9.3,108.3,0.5,9.7,35.1,32.8,120.6,106.9,13.6


In [ ]:
#dropped stats for EST OFF RTG
data_scaled.drop(["NETRTGPerSOS_Per 100_Clutch_Plr", "DEFRTG_Per 100_Clutch_Plr",
                  "NETRTG_Per 100_Clutch_Plr", "DREBPerSOS_Per 100_Clutch_Plr",
                  "DREB_Per 100_Clutch_Plr", '+Per_PerSOS_Per 100_Clutch_Plr',
                  'DD2PerSOS_Per 100_Clutch_Plr', "+Per__Per 100_Clutch_Plr",
                  'OPP PTSPerSOS_Per 100_Clutch', 'oFGA_Above the Break 3', 'Seed',
                  'DREBPerSOS_Per 100_Clutch_Plr', 'oFGA_20_24 FT._Last10'],axis=1,inplace = True)

#Variables Added to EST Off Rtg:
data_correlated_df2.insert(26,"PTS_Per 48",data_scaled["PTS_Per 48"])
data_correlated_df2.insert(27,"EST. OFFRTGPerSOS",data_scaled["EST. OFFRTGPerSOS"])
data_correlated_df2.insert(28,"FGMPerSOS_Corner 3",data_scaled["FGMPerSOS_Corner 3"])
data_correlated_df2.insert(29,"PP100PerSOS_LA",data_scaled["PP100PerSOS_LA"]) #
data_correlated_df2.insert(30,"eFG+_Plr",data_scaled["eFG+_Plr"]) #
data_correlated_df2.insert(31,"FGPerc_10_14 FT._Last10.1", data_scaled["FGPerc_10_14 FT._Last10.1"])
data_correlated_df2.insert(32,"AST RATIO_Per 100_vsOwnConf_Plr", data_scaled["AST RATIO_Per 100_vsOwnConf_Plr"])
data_correlated_df2.insert(33,"FTAPerSOS_Per 100_Clutch", data_scaled["FTAPerSOS_Per 100_Clutch"])
data_correlated_df2.insert(34,"TSPerc_Per 48_Clutch", data_scaled["TSPerc_Per 48_Clutch"])
data_correlated_df2.insert(35,"FGA_<5 FT.",data_scaled["FGA_<5 FT."])
data_correlated_df2.insert(36,"AST_Per 100_vsOwnConf_Plr",data_scaled["AST_Per 100_vsOwnConf_Plr"]) #
data_correlated_df2.insert(37,"FGA_RA",data_scaled["FGA_RA"]) #eFG+_Plr
data_correlated_df2.insert(38,"ASTPerTO_Per 48_Clutch",data_scaled["ASTPerTO_Per 48_Clutch"]) #
data_correlated_df2.insert(39,"AST Ratio_Per 100 Plays_vsOwnC",data_scaled["AST Ratio_Per 100 Plays_vsOwnC"]) #

In [ ]:
#Feature Importance:
scaler = StandardScaler() #MinMaxScaler, StandardScaler
X_train, X_test, y_train = df_NP[10:30], df_NP[0:10], df[10:30]
data_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

data_scaled.drop(["SOS", "SOS_Use", "EFGPercPerSOS_Per 100_Clutch_Plr", "FGPercPerSOS_Per 100_Clutch_Plr",
                  "FGPerc_5_9 FT._Last10.1", "PACE_Per 100_Plr", "ROAD SOS", "FGPercPerSOS_10_14 FT._Last10",
                  "FGAPerSOS_5_9 FT._Last10", 'FGPercPerSOS_Above the Break 3', 'PACE_Per 100_vsOwnConf_Plr',
                  'FGPerc_Right Corner 3', 'FGMPerSOS_Mid Range', 'FGPercPerSOS_20_24 FT._vsOwnC',
                  'WSPer48PerSOS_Per 100_Plr', 'EFGPercPerSOS_Per 100_vsOwnConf_Plr',
                  'TD3_Per 100_Clutch_Plr', 'FGAPerSOS_10_14 FT.', 'FGPercPerSOS_10_14 FT.',
       'FGPercPerSOS_20_24 FT.'],axis=1,inplace = True) #dropped stats for EST DEF RTG

data_correlated = correlation(data_scaled, 0.561, y_train['EST_DEFRTG_Pfs']) #
data_correlated_df = pd.DataFrame(data_correlated)
data_correlated_df2 = data_correlated_df.transpose() #Correlated inputs

#Variables Added to EST Def Rtg:
data_correlated_df2.insert(21,"EST. DEFRTG",data_scaled["EST. DEFRTG"])
data_correlated_df2.insert(22,"OPP EFGPercPerSOS_PerGM",data_scaled["OPP EFGPerc_PerGM"])
data_correlated_df2.insert(23,"OPP TOVPerc_PerGM",data_scaled["OPP TOVPerc_PerGM"])
data_correlated_df2.insert(24,"OPP FBPSPerSOS_Per 100",data_scaled["OPP FBPSPerSOS_Per 100"])
data_correlated_df2.insert(25,"OPP PITPPerSOS_Per 48_Clutch",data_scaled["OPP PITPPerSOS_Per 48_Clutch"])
data_correlated_df2.insert(26,"OPP PITP PerSOS_Per 100",data_scaled["OPP PITP PerSOS_Per 100"])
data_correlated_df2.insert(27,"PP100PerSOS_LA",data_scaled["PP100PerSOS_LA"])

X = df_NP.loc[:, data_correlated_df2.columns] #get non scaled data with important features

#Train test split and scale:
X_train, X_test = X[10:30], X[0:10] #split non-scaled important variables
X_train_processed = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test_processed = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
correlated_scaled_data = pd.merge(X_test_processed, X_train_processed, how = 'outer')
#correlated_scaled_data_MMS = pd.merge(X_test_processed, X_train_processed, how = 'outer')

#PCA:
pca=PCA(n_components = 2)
X_train_processed_PCA = pca.fit_transform(X_train_processed)
X_train_PCA_df = pd.DataFrame(X_train_processed_PCA)
X_test_processed_PCA = pca.transform(X_test_processed)
X_test_PCA_df = pd.DataFrame(X_test_processed_PCA)
data_PCA = pd.merge(X_test_PCA_df, X_train_PCA_df, how = 'outer')
#data_PCA_MMS = pd.merge(X_test_PCA_df, X_train_PCA_df, how = 'outer')

print("Principal axes:\n", pca.components_.tolist())
print("Explained variance:\n", pca.explained_variance_.tolist())
print("Mean:", pca.mean_)

#LLE:
#X_swiss, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=19, random_state=42) #n_components=2 is default, neighbors 5 is default
X_unrolled_train = lle.fit_transform(X_train_processed)
X_train_LLE_df = pd.DataFrame(X_unrolled_train)
X_unrolled_test = lle.transform(X_test_processed)
X_test_LLE_df = pd.DataFrame(X_unrolled_test)
data_LLE = pd.merge(X_test_LLE_df, X_train_LLE_df, how = 'outer')
#data_LLE_19 = pd.merge(X_test_LLE_df, X_train_LLE_df, how = 'outer')
#data_LLE_MMS = pd.merge(X_test_LLE_df, X_train_LLE_df, how = 'outer')
#data_LLE_MMS_19 = pd.merge(X_test_LLE_df, X_train_LLE_df, how = 'outer')

X = correlated_scaled_data #try correlated_scaled_data, data_PCA, data_LLE
y = df2['EST_DEFRTG_Pfs'] #

Principal axes:
 [[-0.08281468915529229, 0.21831977462905658, -0.021991897713202806, -0.017236172657509496, 0.003169004325354521, -0.04750319084704219, 0.04661345691090137, 0.21602618561674564, -0.09536650240716478, -0.1066848202285669, 0.20979238714276652, 0.02881300299345156, 0.18853561191224505, 0.02881300299345156, 0.19658769907292986, 0.18887754684176655, 0.2028519257627902, 0.2285766121679995, 0.07131500094849327, 0.036627311055164584, 0.20344912826521047, 0.014577648322721496, 0.23334785690710796, 0.07537672146902777, 0.20950656833149323, 0.1908486346898339, 0.2233395890887892, 0.2378275009822902, 0.22374022192411858, 0.2285766121679995, -0.12064056011402895, 0.19460257047124227, 0.18291308548824034, 0.23191359738509962, 0.08263070392349028, -0.027020998266235843, 0.15958569942855697, -0.1564239525939719, 0.17259317651115802], [-0.3491920468129267, -0.051539699018980506, -0.11011195542573891, -0.29766542709129307, -0.2623365325774113, -0.3542251212788542, 0.369315028091613, 0.15

In [ ]:
len(data_correlated_df2.columns)

39

In [ ]:
data_correlated_df2.columns

Index(['Point Diff Above AveragePer 100 Plays', 'WinPercPerSOS_Per 100 Plays',
       'Offensive Rating_Per 48_Last 10', 'EFGPerc_Per 48_Last 10',
       'TSPerc_Per 48_Last 10', 'OPP PTS_Per 100', 'OPP Pts Below Avg_Per 100',
       'OPP Pts Above AvgPerSOS_Per 100', 'Opp OREB_Per 100_vsOwnC',
       'OPP PTS_Per 100_vsOwnC', 'OPP PTSPerSOS_Per 100_vsOwnC',
       'OPP 3PPerc_Per 48_Last10', 'OPP BLKPerSOS_Per 48_Last10',
       'OPP 3PPerc_Per 100_Last10', 'OPP PTSPerSOS_Per 100_Last10',
       'STLPerSOS_Per 48_Clutch', '3PPercPerSOS_Per 100_Clutch',
       'PFDPerSOS_Per 100_Clutch', 'DREBPerc_Per 48_Clutch',
       'NETRTGPerSOS_Per 48_Clutch', 'PIEPerSOS_Per 48_Clutch', 'EST. DEFRTG',
       'OPP EFGPercPerSOS_PerGM', 'OPP TOVPerc_PerGM',
       'OPP FBPSPerSOS_Per 100', 'OPP PITPPerSOS_Per 48_Clutch',
       'OPP PITP PerSOS_Per 100', 'PP100PerSOS_LA',
       'OPP PFPerSOS_Per 48_Clutch', 'OPP BLKPerSOS_Per 100_Clutch',
       'OPP 2ND PTS_Per 48_Clutch', 'FGMPerSOS_5_9 FT.', 'F

In [ ]:
#Model: .6
RLE_Model(X, y, "Ridge", X) #
RLE_Model(X, y, "Lasso", X) #
SGD_model(X,y, 1e-3, 0.1, X) #
GBR_model(X,y, .0001, 0.01, 100, X) #
print("5:")
DTR_model(X,y, 100) #default is 100



Avg. Normalized Score:2.6%
Test predictions:
[106.86505824 106.85746736 106.87769619 106.85874945 106.86166423
 106.88097244 106.87936301 106.84335348 106.87845724 106.85933473]
Avg. Normalized Score:5.4%
Difference of avg scores:2.77%

Lasso alpha:  10000000000.0

Avg. Normalized Score:2.6%
Test predictions:
[106.855 106.855 106.855 106.855 106.855 106.855 106.855 106.855 106.855
 106.855]
Avg. Normalized Score:5.4%
Difference of avg scores:2.77%

Avg. Normalized Score:3.1%
Test predictions:
[148.94101874 131.81888676 107.44819559  97.9438936  117.91816872
 125.43237242 130.79742548 115.8517214  109.15012039 104.1286999 ]
Avg. Normalized Score:13.9%
Difference of avg scores:10.79%

Avg. Normalized Score:1.1%
Test predictions:
[107.06289956 110.07346798 109.80189569 109.81842275 106.84391376
 110.07346798 110.07346798 110.3584641  106.89668494 110.0381508 ]
Avg. Normalized Score:4.3%
Difference of avg scores:3.26%

5:
Avg. Normalized Score:2.6%
Test predictions:
[106.855 106.855 106.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.298e-02, tolerance: 1.404e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.063e-02, tolerance: 1.404e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.773e-02, tolerance: 1.404e

In [ ]:
SVM_models(X, y, "rbf", 3, 100, X) #3.7%, 1.9%, corr_MMS 2.3
SVM_models(X, y, "poly", 2.2, 100, X) #3.9%, 1.1%, LLE_19 4.5, 3.9%, 1.5%, PCA_MMS 1.9, 3%, 1.3%, corr_MMS 2.1
SVM_models(X, y, "linear", 2.9, 100, X) #3.2%, .9%, corr, 3.6%, .5%, PCA 5.1, 3.8%, .6%, PCA_MMS 5.1, 3.2%, .9%, corr_MMS
SVM_model(X,y, 0) #

Avg. Normalized Score:2.2%
Test predictions:
[108.97558781 107.17326456 110.32370114 110.36067404 107.65393002
 110.147853   109.11426446 106.84782192 109.57591537 108.95577641]
Avg. Normalized Score:3.9%
Difference of avg scores:1.69%

Avg. Normalized Score:1.8%
Test predictions:
[113.73784416 106.68610718 111.9805403  112.58968981 109.42171171
 113.53201217 113.22314388 108.53146479 111.20189297 110.96088567]
Avg. Normalized Score:3.0%
Difference of avg scores:1.22%

Avg. Normalized Score:2.3%
Test predictions:
[111.27144874 106.66910621 111.61847162 112.09648311 108.42317828
 112.50910027 111.77647031 107.41263082 110.74692827 110.00776014]
Avg. Normalized Score:3.2%
Difference of avg scores:0.91%

Avg. Normalized Score:10.6%
Test predictions:
Avg. Normalized Score:16.4%
Difference of avg scores:5.85%



In [ ]:
SVM_models(X, y, "poly", 3.2, 100, X)
SVM_models(X, y, "linear", 1.9, 100, X)
SVM_models(X, y, "poly", 4.2, 100, X)
SVM_models(X, y, "linear", 3.9, 100, X)

Avg. Normalized Score:2.4%
Test predictions:
[110.16540274 107.00154587 111.19788849 111.25921385 107.61877239
 111.60086972 110.37457087 106.46123607 110.1607609  109.67362789]
Avg. Normalized Score:3.4%
Difference of avg scores:1.03%

Avg. Normalized Score:1.6%
Test predictions:
[116.69451091 107.53234492 113.90268218 115.00370363 111.11848715
 116.11729314 115.9823502  110.59335457 112.20923807 112.06400515]
Avg. Normalized Score:3.6%
Difference of avg scores:2.03%

Avg. Normalized Score:2.5%
Test predictions:
[108.70279839 106.73202406 109.70789727 109.67979331 107.02146971
 109.84645918 109.01999418 105.82625674 108.90265146 108.16763588]
Avg. Normalized Score:4.2%
Difference of avg scores:1.71%

Avg. Normalized Score:2.5%
Test predictions:
[108.91380048 106.82309207 110.48976321 110.56790397 107.09405122
 110.49930846 109.38851163 106.07639671 109.4051116  108.57929049]
Avg. Normalized Score:4.0%
Difference of avg scores:1.47%



(array([108.91380048, 106.82309207, 110.48976321, 110.56790397,
        107.09405122, 110.49930846, 109.38851163, 106.07639671,
        109.4051116 , 108.57929049]),
 array([108.60036949, 108.25904283, 109.40078946, 107.10217381,
        108.79942961, 106.68021011, 105.7828754 , 108.60848491,
        107.07352086, 107.18271048, 107.63805182, 105.80036764,
        105.17721683, 106.47759635, 107.45202689, 108.23507913,
        108.27639736, 103.79983224, 106.93538291, 106.137795  ]),
 array([108.91380048, 106.82309207, 110.48976321, 110.56790397,
        107.09405122, 110.49930846, 109.38851163, 106.07639671,
        109.4051116 , 108.57929049, 108.60036949, 108.25904283,
        109.40078946, 107.10217381, 108.79942961, 106.68021011,
        105.7828754 , 108.60848491, 107.07352086, 107.18271048,
        107.63805182, 105.80036764, 105.17721683, 106.47759635,
        107.45202689, 108.23507913, 108.27639736, 103.79983224,
        106.93538291, 106.137795  ]))

In [ ]:
RLE_Model(X, y, "Elastic", X) #
BR_model(X,y) #
Keras_model(X, y, 200, 120, 60, 30, 20, 15, "adamax") #
#adam, nadam; adamax

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.981e+01, tolerance: 1.596e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


KeyboardInterrupt: 

**Estimated** **Ratings**

In [ ]:
#EST_OFFRTG_Pfs Blender: #3.1%, 1%, 3.0% avg error, .6
est_offrtg = pd.read_csv('EST_OFFRTG Inputs.csv')
scaler = StandardScaler()
train_processed = scaler.fit_transform(X_train)
scaled_data = scaler.transform(est_offrtg)
scaled_data_df = pd.DataFrame(scaled_data)

scaler_MMS = MinMaxScaler()
train_processed_MMS = scaler_MMS.fit_transform(X_train)
scaled_data_MMS = scaler_MMS.transform(est_offrtg)
scaled_data_MMS_df = pd.DataFrame(scaled_data_MMS)

X = correlated_scaled_data
X_corr_MMS = correlated_scaled_data_MMS

preds = SVM_models(X_corr_MMS, y, "poly", 2.1, 100, scaled_data_MMS_df) #3%, 1.3%, corr_MMS
preds2 = SVM_models(X, y, "linear", 2.9, 100, scaled_data_df) #3.2%, .9%, corr
preds3 = SVM_models(X_corr_MMS, y, "linear", 2.9, 100, scaled_data_MMS_df) #3.2%, .9%, corr_MMS

#Get stats on blender
train_preds = (preds[1] + preds2[1] + preds3[1])/3

test_preds = (preds[0] + preds2[0] + preds3[0])/3

print("Blender Train Scores then Test Scores:")
Scores(y[10:30], train_preds, y)
print()
Scores(y[0:10], test_preds, y)
print()

est_off_rtg_predictions = pd.DataFrame()
est_off_rtg_predictions["Predictions"] = (preds[2] + preds2[2] + preds3[2])/3
#est_off_rtg_predictions.to_excel("EST_OFF_RTG_Predictions.xlsx")

Avg. Normalized Score:1.8%
Normalized RMSE:1.8%
Normalized MAE:1.74%
Avg. Error:3.8%
10    2.100332
11    2.100433
12    2.100182
13    2.099778
14    2.099815
15    2.099892
16    0.822862
17    2.100343
18    0.480200
19    2.099811
20    2.099998
21    2.099694
22    2.100189
23    2.099655
24    1.085208
25    2.046758
26    2.100065
27    2.100148
28    2.100131
29    1.752191
Name: EST_OFFRTG_Pfs, dtype: float64
Test predictions:
[114.19043341 106.71098445 112.18064663 112.82069819 109.64403624
 113.7927144  113.49271    108.75371958 111.33032444 111.12741127]
Avg. Normalized Score:3.0%
Normalized RMSE:3.3%
Normalized MAE:2.79%
Avg. Error:3.0%
0    2.509567
1    3.789016
2    0.219353
3    4.720698
4    1.744036
5    1.492714
6    1.592710
7    2.253720
8    4.969676
9    6.972589
Name: EST_OFFRTG_Pfs, dtype: float64
Difference of avg scores:1.29%

Avg. Normalized Score:2.3%
Normalized RMSE:2.3%
Normalized MAE:2.23%
Avg. Error:4.8%
10    2.900416
11    1.253654
12    2.900112
13 

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(


In [ ]:
est_off_rtg_predictions["Predictions"]

0    110.566016
1    111.586163
Name: Predictions, dtype: float64

In [ ]:
#EST_DEFRTG_Pfs Blender: #2.3%, .8%, 2.2% avg error, .561
est_def_rtg = pd.read_csv('EST_DEFRTG Inputs.csv')
scaler = StandardScaler()
train_processed = scaler.fit_transform(X_train)
scaled_data = scaler.transform(est_def_rtg)
scaled_data_df = pd.DataFrame(scaled_data)

lle.fit_transform(X_train)
scaled_data_LLE = lle.transform(scaled_data)
scaled_data_LLE_df = pd.DataFrame(scaled_data_LLE)

scaler_MMS = MinMaxScaler()
scaler_MMS.fit_transform(X_train)
scaled_data_MMS = scaler_MMS.transform(est_def_rtg)
scaled_data_MMS_df = pd.DataFrame(scaled_data_MMS)

X = correlated_scaled_data
#X_PCA = data_PCA
X_corr_MMS = correlated_scaled_data_MMS
#X_PCA_MMS = data_PCA_MMS
X_LLE_19 = data_LLE_19
#X_LLE_19_MMS = data_LLE_MMS_19

loaded_BR_model = pickle.load(open("BR_model_LLE_DEF", "rb")) #

preds = (loaded_BR_model.predict(X_LLE_19[0:10]), loaded_BR_model.predict(X_LLE_19[10:30]), loaded_BR_model.predict(scaled_data_LLE)) #1.7%, .8% (21), LLE_19
preds2 = SVM_models(X, y, "linear", 3.5, 100, scaled_data_df) #3.1%, .8% , corr
#preds3 = SVM_models(X_corr_MMS, y, "linear", 3.5, 100, scaled_data_MMS_df) #3.2%, .9%, corr_MMS
#preds4 = SVM_models(X_PCA_MMS, y, "linear", 4.7, 100, X_PCA_MMS) #3.2%, .6% (35), PCA_MMS

#Get stats on blender
train_preds = (preds[1] + preds2[1])/2

test_preds = (preds[0] + preds2[0])/2

print("Blender Train Scores then Test Scores:")
Scores(y[10:30], train_preds, y)
print()
Scores(y[0:10], test_preds, y)
print()

est_def_rtg_predictions = pd.DataFrame()
est_def_rtg_predictions["Predictions"] = (preds[2] + preds2[2])/2
#est_def_rtg_predictions.to_excel("EST_DEF_RTG_Predictions.xlsx")

Avg. Normalized Score:2.3%
Normalized RMSE:2.4%
Normalized MAE:2.13%
Avg. Error:4.4%
10    0.654068
11    0.193780
12    3.499542
13    2.465323
14    0.600105
15    3.500073
16    3.499927
17    3.500237
18    1.109154
19    2.235120
20    3.500338
21    3.500209
22    1.030045
23    1.168083
24    0.672774
25    2.096131
26    1.509888
27    3.500496
28    3.353054
29    2.146549
Name: EST_DEFRTG_Pfs, dtype: float64
Test predictions:
[105.2145885  104.8934243  102.6361033  102.68941643 103.81295737
 104.0308654  103.73366245 106.2950465  102.36541871 105.55570417]
Avg. Normalized Score:3.1%
Normalized RMSE:3.2%
Normalized MAE:2.88%
Avg. Error:3.0%
0    2.785411
1    5.406576
2    4.663897
3    1.210584
4    2.412957
5    4.569135
6    3.233662
7    2.204953
8    0.434581
9    2.744296
Name: EST_DEFRTG_Pfs, dtype: float64
Difference of avg scores:0.78%

Blender Train Scores then Test Scores:
Avg. Normalized Score:1.5%
Normalized RMSE:1.6%
Normalized MAE:1.38%
Avg. Error:2.8%
10    1.1

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LocallyLinearEmbedding was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(


In [ ]:
est_def_rtg_predictions["Predictions"]

0    98.156553
1    98.668559
Name: Predictions, dtype: float64